In [1]:
from typing import NamedTuple

import kfp
from kfp import dsl
from kfp.components import func_to_container_op, InputPath, OutputPath

kfp.__version__

'1.8.19'

In [2]:
import os

with open(os.environ['KF_PIPELINES_SA_TOKEN_PATH'], "r") as f:
    TOKEN = f.read()

In [6]:
@func_to_container_op
def get_random_int_op(minimum: int, maximum: int) -> int:
    """Generate a random number between minimum and maximum (inclusive)."""
    import random
    result = random.randint(minimum, maximum)
    print(result)
    return result


@func_to_container_op
def flip_coin_op() -> str:
    """Flip a coin and output heads or tails randomly."""
    import random
    result = random.choice(['heads', 'tails'])
    print(result)
    return result


@func_to_container_op
def print_op(message: str):
    """Print a message."""
    print(message)
    

@dsl.pipeline(
    name='Conditional execution pipeline',
    description='Shows how to use dsl.Condition().'
)
def flipcoin_pipeline():
    flip = flip_coin_op()
    with dsl.Condition(flip.output == 'heads'):
        random_num_head = get_random_int_op(0, 9)
        with dsl.Condition(random_num_head.output > 5):
            print_op('heads and %s > 5!' % random_num_head.output)
        with dsl.Condition(random_num_head.output <= 5):
            print_op('heads and %s <= 5!' % random_num_head.output)

    with dsl.Condition(flip.output == 'tails'):
        random_num_tail = get_random_int_op(10, 19)
        with dsl.Condition(random_num_tail.output > 15):
            print_op('tails and %s > 15!' % random_num_tail.output)
        with dsl.Condition(random_num_tail.output <= 15):
            print_op('tails and %s <= 15!' % random_num_tail.output)


# Submit the pipeline for execution:
#kfp.Client(host=kfp_endpoint).create_run_from_pipeline_func(flipcoin_pipeline, arguments={})

# %% [markdown]
# ## Exit handlers
# You can use `with dsl.ExitHandler(exit_task):` context to execute a task when the rest of the pipeline finishes (succeeds or fails)

# %%
@func_to_container_op
def fail_op(message):
    """Fails."""
    import sys
    print(message)    
    sys.exit(1)


@dsl.pipeline(
    name='Conditional execution pipeline with exit handler',
    description='Shows how to use dsl.Condition() and dsl.ExitHandler().'
)
def flipcoin_exit_pipeline():
    exit_task = print_op('Exit handler has worked!')
    with dsl.ExitHandler(exit_task):
        flip = flip_coin_op()
        with dsl.Condition(flip.output == 'heads'):
            random_num_head = get_random_int_op(0, 9)
            with dsl.Condition(random_num_head.output > 5):
                print_op('heads and %s > 5!' % random_num_head.output)
            with dsl.Condition(random_num_head.output <= 5):
                print_op('heads and %s <= 5!' % random_num_head.output)

        with dsl.Condition(flip.output == 'tails'):
            random_num_tail = get_random_int_op(10, 19)
            with dsl.Condition(random_num_tail.output > 15):
                print_op('tails and %s > 15!' % random_num_tail.output)
            with dsl.Condition(random_num_tail.output <= 15):
                print_op('tails and %s <= 15!' % random_num_tail.output)

        with dsl.Condition(flip.output == 'tails'):
            fail_op(message="Failing the run to demonstrate that exit handler still gets executed.")

In [7]:
endpoint = 'http://ml-pipeline.kubeflow:8888'
kfp_client = kfp.Client(host=endpoint, existing_token=TOKEN)

In [8]:
kfp.compiler.Compiler().compile(flipcoin_exit_pipeline, 'test-struct.yaml')

In [10]:
run = kfp_client.create_run_from_pipeline_func(
    flipcoin_exit_pipeline,
    arguments={}
)
url = f'{endpoint}/#/runs/details/{run.run_id}'
print(url)

http://ml-pipeline.kubeflow:8888/#/runs/details/0168c548-e9f2-4e92-aefd-2cb7b7f4fda6


In [4]:
from kfp.v2.dsl import component

In [5]:
@component
def div(a: float, b: float) -> float:
  '''Calculates sum of two arguments'''
  return a / b


@dsl.pipeline(
  name='div-pipeline',
  description='An example pipeline that performs div calculations.',
)
def test_div_pipeline(a: float = 1, b: float = 7):
  div_task = div(a, b)


run = kfp_client.create_run_from_pipeline_func(
    test_div_pipeline,
    arguments={
        'a': 10,
        'b': 5
    },
    mode=kfp.dsl.PipelineExecutionMode.V2_COMPATIBLE,
)
url = f'{endpoint}/#/runs/details/{run.run_id}'
print(url)

/opt/conda/lib/python3.8/site-packages/kfp/compiler/compiler.py:79: UserWarning: V2_COMPATIBLE execution mode is at Beta quality. Some pipeline features may not work as expected.
  warnings.warn('V2_COMPATIBLE execution mode is at Beta quality.'


http://ml-pipeline.kubeflow:8888/#/runs/details/7d56131f-b0a1-4ea2-b304-7e5b2f3ddc1f


In [ ]:
data_loading_task = (
        load_dataset(mount_path)
        .add_volume(k8s_client.V1Volume(name='torch-mnist-datavol'))
        .add_volume_mount(
            k8s_client.V1VolumeMount(
                mount_path=mount_path,
                name='torch-mnist-datavol',
            )
        )
    )